<a href="https://colab.research.google.com/github/meizhong986/WhisperJAV/blob/main/notebook/WhisperJAV_colab_parallel_expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/meizhong986/WhisperJAV/blob/main/notebook/WhisperJAV_colab_parallel_expert.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/></a>

# WhisperJAV Ensemble Dual-GPU Edition (v1.8.0)

**Universal Parallel Workflow**

| Platform | Logic | Storage |
|----------|-------|---------|
| **Kaggle** | **Parallel** (2x T4) | **Input**: `/kaggle/input` (Dataset) <br> **Output**: `/kaggle/working` (Artifacts) |
| **Colab** | **Sequential** (1x GPU) | **Input/Output**: Google Drive |

---
### **Workflow**
1. **Configure**: Select your settings, models, and audio preferences.
2. **Setup**: Installs dependencies and prepares the environment (**Run Once**).
3. **Transcribe**: Processes your video files using the configured settings.
4. **Translate**: (Optional) Uses AI to translate subtitles to English.

<small><i>Tip: Select your Dataset in Kaggle, then use "Run All". The notebook is designed to Fail Fast if resources are missing.</i></small>

In [ ]:
#@title Step 1: Expert Configuration { display-mode: "form" }

#@markdown ## 📁 Files & Output
folder_name = "WhisperJAV" #@param {type:"string"}
subtitle_language = "Japanese" #@param ["Japanese", "English (auto-translate)", "English (AI translate)"]
spoken_language = "Japanese" #@param ["Japanese", "Chinese", "English", "Korean"]

#@markdown ---
#@markdown ## 1️⃣ Pass 1 Configuration (GPU 0)
pass1_quality = "balanced" #@param ["faster", "fast", "balanced", "fidelity", "transformers"]
pass1_sensitivity = "aggressive" #@param ["conservative", "balanced", "aggressive"]
pass1_model = "large-v2" #@param ["automatic", "large-v2", "large-v3", "turbo", "kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"]

#@markdown **Expert Audio Setup (Pass 1)**
pass1_scene_detector = "semantic" #@param ["automatic", "auditok", "silero", "semantic"]
pass1_speech_segmenter = "ten" #@param ["automatic", "silero", "ten", "none"]
pass1_speech_enhancer = "none" #@param ["none", "ffmpeg-dsp", "clearvoice", "zipenhancer", "bs-roformer"]
#@markdown <font size="1">auditok=energy (fast), silero=VAD, semantic=texture (complex audio) | enhancer: ffmpeg-dsp(no GPU), clearvoice(48k), bs-roformer(vocal)</font>

#@markdown **FFmpeg Filters (Pass 1)** *(only if enhancer is ffmpeg-dsp)*
pass1_ffmpeg_amplify = True #@param {type:"boolean"}
pass1_ffmpeg_loudnorm = False #@param {type:"boolean"}
pass1_ffmpeg_compress = False #@param {type:"boolean"}
pass1_ffmpeg_highpass = False #@param {type:"boolean"}

#@markdown ---
#@markdown ## 2️⃣ Pass 2 Configuration (GPU 1)
pass2_quality = "fidelity" #@param ["faster", "fast", "balanced", "fidelity", "transformers"]
pass2_sensitivity = "balanced" #@param ["conservative", "balanced", "aggressive"]
pass2_model = "turbo" #@param ["automatic", "large-v2", "large-v3", "turbo", "kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"]

#@markdown **Expert Audio Setup (Pass 2)**
pass2_scene_detector = "semantic" #@param ["automatic", "auditok", "silero", "semantic"]
pass2_speech_segmenter = "ten" #@param ["automatic", "silero", "ten", "none"]
pass2_speech_enhancer = "ffmpeg-dsp" #@param ["none", "ffmpeg-dsp", "clearvoice", "zipenhancer", "bs-roformer"]

#@markdown **FFmpeg Filters (Pass 2)** *(only if enhancer is ffmpeg-dsp)*
pass2_ffmpeg_amplify = True #@param {type:"boolean"}
pass2_ffmpeg_loudnorm = False #@param {type:"boolean"}
pass2_ffmpeg_compress = False #@param {type:"boolean"}
pass2_ffmpeg_highpass = False #@param {type:"boolean"}

#@markdown ---
#@markdown ## 🔗 Merge Strategy
merge_method = "prefer first pass" #@param ["automatic", "keep all", "prefer first pass", "prefer second pass"]

#@markdown ---
#@markdown ## 🤖 AI Translation *(if selected)*
translation_service = "deepseek" #@param ["deepseek", "openrouter", "gemini", "claude", "gpt"]
api_key = "" #@param {type:"string"}
translation_style = "standard" #@param ["standard", "explicit"]

#@markdown ---
#@markdown ## ⚙️ Session
opening_credit = "" #@param {type:"string"}
closing_credit = "Subs by WhisperJAV" #@param {type:"string"}
auto_disconnect = True #@param {type:"boolean"}

# ═══════════════════════════════════════════
# CONFIGURATION LOGIC
# ═══════════════════════════════════════════

# Mapping dictionaries
combine_map = {"automatic": "smart_merge", "keep all": "full_merge",
               "prefer first pass": "pass1_primary", "prefer second pass": "pass2_primary"}
language_map = {"Japanese": "native", "English (auto-translate)": "direct-to-english",
                "English (AI translate)": "llm"}
tone_map = {"standard": "standard", "explicit": "pornify"}
spoken_language_map = {"Japanese": "japanese", "Chinese": "chinese", "English": "english", "Korean": "korean"}

# Model mapping (None = use pipeline default)
model_map = {
    "automatic": None,
    "large-v2": "large-v2",
    "large-v3": "large-v3",
    "turbo": "turbo",
    "kotoba-bilingual": "kotoba-tech/kotoba-whisper-bilingual-v1.0",
    "kotoba-v2.0": "kotoba-tech/kotoba-whisper-v2.0",
    "kotoba-v2.1": "kotoba-tech/kotoba-whisper-v2.1",
    "kotoba-v2.2": "kotoba-tech/kotoba-whisper-v2.2"
}

# Define model compatibility:
KOTOBA_MODELS = {"kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"}
LEGACY_PIPELINES = {"faster", "fast", "balanced", "fidelity"}

# Auto-correct incompatible model-pipeline combinations
warnings_list = []

# Check Pass 1 compatibility
if pass1_model in KOTOBA_MODELS and pass1_quality in LEGACY_PIPELINES:
    warnings_list.append(f"Pass 1: {pass1_model} requires 'transformers' pipeline. Auto-correcting from '{pass1_quality}' to 'transformers'.")
    pass1_quality = "transformers"

# Check Pass 2 compatibility
if pass2_model in KOTOBA_MODELS and pass2_quality in LEGACY_PIPELINES:
    warnings_list.append(f"Pass 2: {pass2_model} requires 'transformers' pipeline. Auto-correcting from '{pass2_quality}' to 'transformers'.")
    pass2_quality = "transformers"

# Memory warning
heavy_enhancers = {'clearvoice', 'bs-roformer', 'zipenhancer'}
if pass1_speech_enhancer in heavy_enhancers and pass2_speech_enhancer in heavy_enhancers:
    warnings_list.append("Using GPU-based enhancement on both passes may cause OOM on T4 GPU (Sequential Mode). Suggest using ffmpeg-dsp for one pass.")

# Helpers
def build_ffmpeg_filters(amplify, loudnorm, compress, highpass):
    """Combine selected FFmpeg filters into comma-separated string."""
    filters = []
    if amplify: filters.append("amplify")
    if loudnorm: filters.append("loudnorm")
    if compress: filters.append("compress")
    if highpass: filters.append("highpass")
    return ",".join(filters) if filters else None

def map_value(val):
    return None if val == "automatic" else val

def map_segmenter(val):
    return "none" if val == "none" else map_value(val)

# Unified Config Construction
WHISPERJAV_CONFIG = {
    'pass1_pipeline': pass1_quality,
    'pass1_sensitivity': pass1_sensitivity,
    'pass1_speech_segmenter': map_segmenter(pass1_speech_segmenter),
    'pass1_model': model_map[pass1_model],
    'pass2_pipeline': pass2_quality,
    'pass2_sensitivity': pass2_sensitivity,
    'pass2_speech_segmenter': map_segmenter(pass2_speech_segmenter),
    'pass2_model': model_map[pass2_model],
    'merge_strategy': combine_map[merge_method],
    'folder_name': folder_name,
    'subtitle_language': language_map[subtitle_language],
    'language': spoken_language_map[spoken_language],
    'translation_service': translation_service,
    'api_key': api_key,
    'translation_style': tone_map[translation_style],
    'opening_credit': opening_credit,
    'closing_credit': closing_credit,
    'auto_disconnect': auto_disconnect,
    # Compatibility checks for Step 2
    '_pass1_quality': pass1_quality,
    '_pass1_sensitivity': pass1_sensitivity,
    '_pass1_speech_segmenter': pass1_speech_segmenter,
    '_pass1_model': pass1_model,
    '_pass2_quality': pass2_quality,
    '_pass2_sensitivity': pass2_sensitivity,
    '_pass2_speech_segmenter': pass2_speech_segmenter,
    '_pass2_model': pass2_model,
    '_merge_method': merge_method,
    '_subtitle_language': subtitle_language,
    '_translation_style': translation_style,
}

WHISPERJAV_EXPERT_CONFIG = {
    # Pass 1 Expert
    'pass1_scene_detector': map_value(pass1_scene_detector),
    'pass1_speech_segmenter': map_segmenter(pass1_speech_segmenter), # Now unified
    'pass1_speech_enhancer': None if pass1_speech_enhancer == "none" else pass1_speech_enhancer,
    'pass1_ffmpeg_filters': build_ffmpeg_filters(pass1_ffmpeg_amplify, pass1_ffmpeg_loudnorm, pass1_ffmpeg_compress, pass1_ffmpeg_highpass) if pass1_speech_enhancer == "ffmpeg-dsp" else None,
    # Pass 2 Expert
    'pass2_scene_detector': map_value(pass2_scene_detector),
    'pass2_speech_segmenter': map_segmenter(pass2_speech_segmenter), # Now unified
    'pass2_speech_enhancer': None if pass2_speech_enhancer == "none" else pass2_speech_enhancer,
    'pass2_ffmpeg_filters': build_ffmpeg_filters(pass2_ffmpeg_amplify, pass2_ffmpeg_loudnorm, pass2_ffmpeg_compress, pass2_ffmpeg_highpass) if pass2_speech_enhancer == "ffmpeg-dsp" else None,
    # Display helpers
    '_pass1_scene_detector': pass1_scene_detector,
    '_pass1_speech_enhancer': pass1_speech_enhancer,
    '_pass2_scene_detector': pass2_scene_detector,
    '_pass2_speech_enhancer': pass2_speech_enhancer,
}

from IPython.display import display, HTML

# Display warnings
for warning in warnings_list:
    display(HTML(f'<div style="padding:6px 10px;background:#fef9c3;border-radius:4px;font-size:10px;margin-bottom:4px"><b>⚠️ Auto-corrected:</b> {warning}</div>'))

# Build status display
p1_info = f"{pass1_quality}"
if pass1_speech_segmenter != "automatic":
    p1_info += f"/{pass1_speech_segmenter}"
if pass1_model != "automatic":
    p1_info += f"/{pass1_model}"

p2_info = f"{pass2_quality}"
if pass2_speech_segmenter != "automatic":
    p2_info += f"/{pass2_speech_segmenter}"
if pass2_model != "automatic":
    p2_info += f"/{pass2_model}"

display(HTML(f'<div style="padding:10px;background:#e0f2fe;border-radius:4px;font-size:11px">'
             f'<b>Parallel Configuration Loaded</b><br>'
             f'Pass 1: {p1_info} | Pass 2: {p2_info}<br>'
             f'Merge: {merge_method} | Folder: {folder_name} | Language: {spoken_language}'
             f'</div>'))

In [ ]:
#@title Step 2: Setup & Environment (Run Once) { display-mode: "form" }
#@markdown - **Fails Fast** if GPU or Internet is missing.
#@markdown - Installs **System Tools** (FFmpeg, libc++) and **Python Libraries** (WhisperJAV, VADs).
#@markdown - Prepares the environment for all subsequent steps.

import importlib.util
import os
import shutil
import subprocess
import sys
import time

FAIL_FAST = True

def section(title):
    print(f"\n{'─'*40}\n{title}\n{'─'*40}")

def status(msg, ok=True):
    print(f"{'✓' if ok else '✗'} {msg}")

def must(condition, msg):
    if not condition:
        if FAIL_FAST:
            raise RuntimeError(f"SETUP FAILED: {msg}")
        print(f"WARN: {msg}")

def run_shell(name, cmd):
    """Run a shell command (apt, etc)."""
    print(f"... installing {name}")
    r = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if r.returncode != 0:
        print((r.stderr or r.stdout or "")[-2000:])
        raise RuntimeError(f"Install failed: {name}")
    status(f"Installed {name}")

def run_pip(name, packages):
    """Run pip install safely (avoiding shell injection)."""
    print(f"... installing {name}")
    # cmd: python -m pip install -q --upgrade [packages...]
    cmd = [sys.executable, "-m", "pip", "install", "-q"] + packages
    r = subprocess.run(cmd, check=False, text=True, capture_output=True)
    if r.returncode != 0:
        print(f"--- pip stderr for {name} ---")
        print((r.stderr or r.stdout or "")[-2000:])
        raise RuntimeError(f"Pip Install failed: {name}")
    status(f"Installed {name}")

# ═══════════════════════════════════════════
# 1. PRE-FLIGHT CHECKS (Fail Fast)
# ═══════════════════════════════════════════
section("PRE-FLIGHT CHECKS")

# Detect Platform
if "google.colab" in sys.modules:
    PLATFORM = "colab"
elif os.path.exists("/kaggle"):
    PLATFORM = "kaggle"
else:
    PLATFORM = "local"
print(f"Platform: {PLATFORM.upper()}")
print(f"Python:   {sys.executable} ({sys.version.split()[0]})")

# Check GPU
gpu_check = subprocess.run("nvidia-smi --query-gpu=name --format=csv,noheader", shell=True, capture_output=True, text=True)
must(gpu_check.returncode == 0 and bool(gpu_check.stdout.strip()), "No GPU detected. Switch runtime to GPU.")
gpus = [line.strip() for line in gpu_check.stdout.splitlines() if line.strip()]
status(f"GPU(s) Detected: {len(gpus)} ({', '.join(gpus)})")

# Check Internet (Required for install)
try:
    import socket
    socket.create_connection(("www.google.com", 80), timeout=2.0).close()
    status("Internet reachable")
except OSError:
    must(False, "Internet disabled/unreachable. Cannot install dependencies.")

# ═══════════════════════════════════════════
# 2. INSTALLATION
# ═══════════════════════════════════════════
section("INSTALLING DEPENDENCIES (3-5 min)")
install_start = time.time()

# Determine sudo usage
prefix = ""
if shutil.which("sudo") and not (hasattr(os, "geteuid") and os.geteuid() == 0):
    prefix = "sudo "

# A. System Layer (APT)
# Unconditional: libraries for Audio (portaudio), Video (ffmpeg), and VAD (libc++)
sys_pkgs = "ffmpeg portaudio19-dev libc++1 libc++abi1"
run_shell("System Tools (apt)", f"{prefix}apt-get update -qq && {prefix}apt-get install -y -qq {sys_pkgs}")

# B. Core Acceleration Layer (Pip)
# Upgrade pip first
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "pip"], check=False)

# Only install heavy/complex libs here. Let WhisperJAV pull the rest.
core_libs = [
    # Basic Utils
    "tqdm", "numba", "tiktoken", "soundfile", "auditok", "requests", "colorama", "regex",
    "numpy>=2.0", "scipy>=1.10.1", "librosa>=0.11.0",
    # ML Stack (Pre-install to ensure GPU versions/cache hits)
    "faster-whisper>=1.1.0", "transformers", "optimum", "accelerate", "huggingface-hub>=0.25.0",
    # Specific VADs (Binary wheels)
    "ten-vad", "silero-vad>=6.0", "pydub",
    # Enhancer bases
    "modelscope>=1.20", "onnxruntime>=1.16.0"
]
run_pip("Core Acceleration Libs", core_libs)

# C. Application Layer (Git)
# These will auto-resolve dependencies like pydantic, pysubtrans, etc.
git_pkgs = [
    ("ffmpeg-python", "git+https://github.com/kkroening/ffmpeg-python.git"),
    ("Whisper", "git+https://github.com/openai/whisper.git@main"),
    ("Stable-TS", "git+https://github.com/meizhong986/stable-ts-fix-setup.git@main"),
    ("ClearVoice", "git+https://github.com/meizhong986/ClearerVoice-Studio.git#subdirectory=clearvoice"),
    ("WhisperJAV", "git+https://github.com/meizhong986/WhisperJAV.git@main"),
]

for name, url in git_pkgs:
    run_pip(name, ["--no-deps", url])

status(f"Installation Complete ({time.time() - install_start:.0f}s)")

# ═══════════════════════════════════════════
# 3. VERIFICATION
# ═══════════════════════════════════════════
section("VERIFICATION")

# Verify Import
try:
    import whisperjav
    status(f"WhisperJAV {whisperjav.__version__} Ready")
except ImportError as e:
    raise RuntimeError(f"Installation successful but import failed: {e}")

# Verify TEN/Libc++ (Common Failure Point)
try:
    import ten_vad
    status("TEN VAD Ready")
except ImportError:
    status("TEN VAD Warning (Import failed)", False)

# Stamp Success
WHISPERJAV_SETUP_COMPLETE = {
    "timestamp": time.time(),
    "platform": PLATFORM,
    "gpu_count": len(gpus)
}
print("\n✓ Environment Ready. Go to Step 3.")

In [ ]:
#@title Step 3: Execution (Transcribe) { display-mode: "form" }
#@markdown - Auto-detects **Files** (Kaggle Dataset or Colab Drive).
#@markdown - Auto-detects **Parallel** (2x GPU) or **Sequential** mode.
#@markdown - Runs the configured WhisperJAV pipeline.

import os
import shutil
import sys
import time
import subprocess
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import HTML, FileLink, display

# ═══════════════════════════════════════════
# CHECKS & PREP
# ═══════════════════════════════════════════
if "WHISPERJAV_SETUP_COMPLETE" not in globals():
    display(HTML('<div style="background:#fee2e2;color:#991b1b;padding:10px;border-radius:4px;"><b>🛑 Setup Incomplete</b><br>Please run <b>Step 2: Setup</b> first.</div>'))
    raise RuntimeError("Step 2 required")

if "WHISPERJAV_CONFIG" not in globals():
    display(HTML('<div style="background:#fee2e2;color:#991b1b;padding:10px;border-radius:4px;"><b>🛑 Config Missing</b><br>Please run <b>Step 1: Configuration</b> first.</div>'))
    raise RuntimeError("Step 1 required")

cfg = WHISPERJAV_CONFIG
expert = globals().get("WHISPERJAV_EXPERT_CONFIG", {})
PLATFORM = WHISPERJAV_SETUP_COMPLETE["platform"]
PARALLEL_MODE = WHISPERJAV_SETUP_COMPLETE["gpu_count"] >= 2

def status(msg, ok=True):
    print(f"{'✓' if ok else '✗'} {msg}")

# ═══════════════════════════════════════════
# I/O SETUP
# ═══════════════════════════════════════════
print(f"Mode: {'PARALLEL (2x GPU)' if PARALLEL_MODE else 'SEQUENTIAL (Ensemble)'}")

if PLATFORM == "colab":
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    input_dir = Path("/content/drive/MyDrive") / cfg["folder_name"]
    output_dir = input_dir
    input_dir.mkdir(parents=True, exist_ok=True)
elif PLATFORM == "kaggle":
    # Search for video inputs
    video_types = {".mp4", ".mkv", ".avi", ".mov", ".wmv", ".webm"}
    input_dir = None
    for p in [Path(f"/kaggle/input/{cfg['folder_name']}"), Path("/kaggle/input")]:
        if p.exists() and any(f.suffix.lower() in video_types for f in p.rglob("*")):
            input_dir = p
            break
    if not input_dir:
        input_dir = Path("/kaggle/input")
        print("WARN: No specific input folder found. Scanning root /kaggle/input.")
    
    output_dir = Path(f"/kaggle/working/{cfg['folder_name']}")
    output_dir.mkdir(parents=True, exist_ok=True)
    os.environ["TMPDIR"] = "/kaggle/working/temp" # Avoid RAM disk overflow
else:
    input_dir = Path(cfg["folder_name"]).absolute()
    output_dir = Path(f"{cfg['folder_name']}_output").absolute()
    output_dir.mkdir(parents=True, exist_ok=True)

status(f"Input: {input_dir}")
status(f"Output: {output_dir}")
WHISPERJAV_OUTPUT_DIR = output_dir # Export for Step 4

# ═══════════════════════════════════════════
# EXECUTION LOGIC
# ═══════════════════════════════════════════
media_files = sorted([p for p in input_dir.rglob("*") if p.suffix.lower() in {".mp4", ".mkv", ".avi",".mov",".wmv",".flv",".webm",".m3v",".mp3",".wav",".flac"}])
if not media_files:
    raise RuntimeError(f"No media files found in {input_dir}")

logs_dir = output_dir / "logs"
logs_dir.mkdir(parents=True, exist_ok=True)
final_outputs = []

def build_cmd(video_path, out_path, pass_cfg_num=None):
    """Builds the CLI command. If pass_cfg_num is set, builds a single-pass command (for parallel)."""
    cmd = [sys.executable, "-m", "whisperjav.main", str(video_path), "--output-dir", str(out_path)]
    cmd += ["--subs-language", "direct-to-english" if cfg["subtitle_language"] == "direct-to-english" else "native"]
    if cfg.get("language"): cmd += ["--language", cfg["language"]]
    
    # Helper to clean args
    def add_arg(name, val):
        if val and val != "none": cmd.extend([name, str(val)])

    if pass_cfg_num:
        # Single Pass Mode (Parallel Worker)
        p = f"pass{pass_cfg_num}"
        cmd += ["--mode", cfg[f"{p}_pipeline"], "--sensitivity", cfg[f"{p}_sensitivity"]]
        add_arg("--model", cfg.get(f"{p}_model"))
        add_arg("--speech-segmenter", cfg.get(f"{p}_speech_segmenter"))
        # Single pass expert args (enhancers/scene) would be mapped here if supported by main CLI in single mode
    else:
        # Ensemble Mode (Sequential/Full)
        cmd += ["--ensemble", "--merge-strategy", cfg["merge_strategy"]]
        for p in ["pass1", "pass2"]:
            cmd += [f"--{p}-pipeline", cfg[f"{p}_pipeline"], f"--{p}-sensitivity", cfg[f"{p}_sensitivity"]]
            add_arg(f"--{p}-model", cfg.get(f"{p}_model"))
            add_arg(f"--{p}-speech-segmenter", cfg.get(f"{p}_speech_segmenter"))
            # Expert
            add_arg(f"--{p}-scene-detector", expert.get(f"{p}_scene_detector"))
            enh = expert.get(f"{p}_speech_enhancer")
            if enh == "ffmpeg-dsp": enh = f"ffmpeg-dsp:{expert.get(f'{p}_ffmpeg_filters', 'amplify')}"
            add_arg(f"--{p}-speech-enhancer", enh)
            
    return cmd

start_time = time.time()
print(f"\nProcessing {len(media_files)} files...")

if PARALLEL_MODE:
    # 2-GPU Parallel Execution: Split Passes, Then Merge
    from whisperjav.ensemble.merge import MergeEngine
    merger = MergeEngine()
    
    for vid in media_files:
        print(f"\n[Parallel] Processing: {vid.name}")
        base = output_dir / vid.stem
        p1_dir, p2_dir = base / "pass1", base / "pass2"
        p1_dir.mkdir(parents=True, exist_ok=True)
        p2_dir.mkdir(parents=True, exist_ok=True)

        def run_pass(num, gpu, out):
            env = os.environ.copy()
            env["CUDA_VISIBLE_DEVICES"] = str(gpu)
            log = logs_dir / f"{vid.stem}_pass{num}.log"
            with log.open("w") as f:
                c = build_cmd(vid, out, pass_cfg_num=num)
                r = subprocess.run(c, env=env, stdout=f, stderr=subprocess.STDOUT, text=True)
            return r.returncode == 0, list(out.glob("*.srt"))[0] if list(out.glob("*.srt")) else None

        with ThreadPoolExecutor(max_workers=2) as exc:
            f1 = exc.submit(run_pass, 1, 0, p1_dir)
            f2 = exc.submit(run_pass, 2, 1, p2_dir)
            ok1, srt1 = f1.result()
            ok2, srt2 = f2.result()

        if ok1 and srt1 and ok2 and srt2:
            merged = output_dir / srt1.name
            merger.merge(srt1, srt2, merged, strategy=cfg["merge_strategy"])
            status(f"Merged: {merged.name}")
            final_outputs.append(merged)
        elif (ok1 and srt1) or (ok2 and srt2):
            winner = srt1 if (ok1 and srt1) else srt2
            dest = output_dir / winner.name
            shutil.copy(winner, dest)
            status(f"Partial Success (One pass failed): {dest.name}")
            final_outputs.append(dest)
        else:
            status(f"Failed: {vid.name}", False)

else:
    # Sequential Execution (Colab/Single-GPU)
    # Run bulk command on directory
    cmd = build_cmd(input_dir, output_dir)
    log_path = logs_dir / "whisperjav_exec.log"
    print("Running sequential ensemble...")
    with log_path.open("w") as f:
        rc = subprocess.run(cmd, stdout=f, stderr=subprocess.STDOUT)
    
    new_srts = list(output_dir.glob("*.srt")) # Logic simplification: grab all SRTs in output
    if rc.returncode == 0 and new_srts:
        status(f"Processed {len(new_srts)} files")
        final_outputs = new_srts
    else:
        status("Execution failed or no SRTs generated. See logs.", False)

WHISPERJAV_NEW_SRTS = final_outputs

# ═══════════════════════════════════════════
# PACKAGING
# ═══════════════════════════════════════════
if final_outputs:
    zip_path = output_dir / f"{cfg['folder_name']}_results.zip"
    import zipfile
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        for f in output_dir.rglob("*"):
            if f.suffix in {".srt", ".json", ".log"}:
                zf.write(f, f.relative_to(output_dir))
    
    print(f"\n📦 Bundle created: {zip_path.name}")
    if PLATFORM == "kaggle":
        display(FileLink(str(zip_path.relative_to(Path.cwd())), result_html_prefix="<b>⬇ Download Results: </b>"))
    elif PLATFORM == "colab":
        try:
            from google.colab import files
            files.download(str(zip_path))
        except: pass

print(f"\nTime: {(time.time()-start_time)/60:.1f} min")
print("Done. Run Step 4 for Translation if needed.")

In [ ]:
#@title Step 4: AI Translation (Optional) { display-mode: "form" }
#@markdown - Translates result SRTs to English using LLMs (GPT, Claude, DeepSeek, etc.).
#@markdown - Requires an API key (or Kaggle Secret).

import os
import sys
import subprocess
from pathlib import Path
from IPython.display import HTML, display, FileLink

def status(msg, ok=True):
    print(f"{'✓' if ok else '✗'} {msg}")

# 1. Validation
if "WHISPERJAV_NEW_SRTS" not in globals() or not WHISPERJAV_NEW_SRTS:
    display(HTML('<div style="background:#f0f9ff;padding:10px;"><b>ℹ No Inputs</b><br>No new subtitles found from Step 3 to translate.</div>'))
    raise RuntimeError("No input files")

if WHISPERJAV_CONFIG["subtitle_language"] != "llm":
    display(HTML('<div style="background:#f0f9ff;padding:10px;"><b>ℹ Translation Skipped</b><br>Select "English (AI translate)" in Step 1 to enable this.</div>'))
    raise RuntimeError("Skipped by config")

# 2. API Key Setup
service = WHISPERJAV_CONFIG["translation_service"]
user_key = WHISPERJAV_CONFIG.get("api_key")
final_key = None

if user_key:
    final_key = user_key
elif os.path.exists("/kaggle"):
    from kaggle_secrets import UserSecretsClient
    try:
        final_key = UserSecretsClient().get_secret(f"{service.upper()}_API_KEY")
    except: pass

if not final_key:
    raise RuntimeError(f"Missing API Key for {service}. Add to Config or Kaggle Secrets.")

# Set Env Var
env_map = {"deepseek": "DEEPSEEK_API_KEY", "openrouter": "OPENROUTER_API_KEY", "gpt":"OPENAI_API_KEY", "claude":"ANTHROPIC_API_KEY", "gemini":"GEMINI_API_KEY"}
os.environ[env_map.get(service, "API_KEY")] = final_key

# 3. Execution
print(f"Translating {len(WHISPERJAV_NEW_SRTS)} files with {service}...")
translated = []

for srt in WHISPERJAV_NEW_SRTS:
    cmd = [sys.executable, "-m", "whisperjav.translate.cli", "-i", str(srt), "--provider", service, "--tone", WHISPERJAV_CONFIG["translation_style"]]
    r = subprocess.run(cmd, capture_output=True, text=True)
    if r.returncode == 0 and r.stdout.strip():
        out_file = Path(r.stdout.strip())
        status(f"Translated: {out_file.name}")
        translated.append(out_file)
    else:
        status(f"Failed: {srt.name} ({r.stderr[:200]})", False)

# 4. Bundle
if translated:
    out_dir = WHISPERJAV_OUTPUT_DIR
    zip_path = out_dir / f"translated_{service}.zip"
    import zipfile
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        for t in translated: zf.write(t, t.name)
        
    print(f"\n📦 Translations Bundle: {zip_path.name}")
    if os.path.exists("/kaggle"):
         display(FileLink(str(zip_path.relative_to(Path.cwd())), result_html_prefix="<b>⬇ Download Translations: </b>"))
    elif "google.colab" in sys.modules:
        from google.colab import files
        files.download(str(zip_path))